In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm



In [ ]:


# Import data

options = pd.read_csv('C:/Users/User/Desktop/Data speciale/SPX_options_data.csv')




In [ ]:

options.head()

In [ ]:
# 3M US Treasury Bill rate (r)
interst_rates = pd.read_csv('C:/Users/User/Desktop/Data speciale/3M_US_yield_data.csv', parse_dates=True, index_col='DATE', na_values='.')
interst_rates = pd.DataFrame(interst_rates['DTB3'])    
interst_rates.dropna(inplace=True)
interst_rates['rate'] = interst_rates['DTB3']/100
del(interst_rates['DTB3'])



In [ ]:
# Underlying asset price (S)

stock_price = pd.read_csv('C:/Users/User/Desktop/Data speciale/SPX_indicies_data.csv', parse_dates=True, index_col='date')
stock_price = pd.DataFrame(stock_price['close'])

In [ ]:
# Volatility (sigma)

vol = pd.read_csv('C:/Users/User/Desktop/Data speciale/SPX_volatility.csv', parse_dates=True, index_col='date')


In [ ]:
# select only the columns we need 

options = options[options['volume'] > 0]

data = options[['date','exdate','strike_price','best_bid','best_offer','optionid']]

# Data cleaning

data['strike_price'] = data['strike_price']/1000
data = data[data['best_bid'] > 0]
data['price'] = (data['best_bid'] + data['best_offer'])/2
data['date'] = pd.to_datetime(data['date'], format = '%Y-%m-%d') 
data['exdate'] = pd.to_datetime(data['exdate'], format = '%Y-%m-%d')
data['time_to_maturity'] = (data['exdate'] - data['date']).dt.days/365
data = data.set_index('date')
data = data[(data['time_to_maturity'] <= 2)]
data = data[(data['time_to_maturity'] > 1/365)]
data = data[(data['price'] >= 0.1)]
data['Year'] = data.index.year
data = data.merge(stock_price, left_index = True, right_index = True, how='left')
data = data.merge(interst_rates, left_index= True, right_index = True, how='left')
data = data.merge(vol, left_index = True, right_index = True, how='left')
data['moneyness'] = data['close']/data['strike_price']
data = data[(data['moneyness'] <= 1.5)]
data = data[(data['moneyness'] >= 0.8)]
#data = data[data['price'] >= (data['close'] - data['strike_price'] * np.exp(-data['rate']*data['time_to_maturity']))] # arbitrage condition
data.dropna(inplace=True)


In [ ]:
# calculating the black scholes price for the options

def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return price


data['BS (vol10)'] = black_scholes_call(data['close'], data['strike_price'], data['time_to_maturity'], data['rate'], data['HistVol10'])
data['BS (vol30)'] = black_scholes_call(data['close'], data['strike_price'], data['time_to_maturity'], data['rate'], data['HistVol30'])
data['BS (vol60)'] = black_scholes_call(data['close'], data['strike_price'], data['time_to_maturity'], data['rate'], data['HistVol60'])
data['BS GARCH'] = black_scholes_call(data['close'], data['strike_price'], data['time_to_maturity'], data['rate'], data['GARCH'])
data['BS VIX'] = black_scholes_call(data['close'], data['strike_price'], data['time_to_maturity'], data['rate'], data['VIX'])

In [ ]:

# data export

df = pd.DataFrame()

df['Stock (S)'], df['Strike (K)'], df['Moneyness (M)'], df['Time to maturity (T)'], df['Risk free rate (r)'] = data['close'], data['strike_price'], data['moneyness'], data['time_to_maturity'], data['rate']
df['BS (vol10)'] = data['BS (vol10)']/df['Strike (K)']
df['BS (vol30)'] = data['BS (vol30)']/df['Strike (K)']
df['BS (vol60)'] = data['BS (vol60)']/df['Strike (K)']
df['BS GARCH'] = data['BS GARCH']/df['Strike (K)']
df['BS VIX'] = data['BS VIX']/df['Strike (K)']
df['vol10'], df['vol30'], df['vol60'], df['GARCH'], df['VIX'] = data['HistVol10'], data['HistVol30'], data['HistVol60'], data['GARCH'], data['VIX']
df['Option Price (C)'] = data['price']/df['Strike (K)']
df['ID'] = data['optionid']
df['Year'] = data['Year']




In [ ]:


# Load the dataset
goyal = pd.read_csv('C:/Users/User/Desktop/Data speciale/Goyal_macrovariables.csv', parse_dates=True, index_col='date')

# Ensure the index is in datetime format (assuming date format is 'YYYYMM')
goyal.index = pd.to_datetime(goyal.index, format='%Y%m')

# Set the index to the last day of the month
goyal.index = goyal.index + pd.offsets.MonthEnd(0)

# Create a new daily date range covering the full period of the dataset
daily_index = pd.date_range(start=goyal.index.min(), end=goyal.index.max(), freq='D')

# Reindex the dataset to the new daily frequency
goyal_daily = goyal.reindex(daily_index)

# Linearly interpolate the 'tchi' and 'shtint' columns to fill in the daily data
goyal_daily['tchi'] = goyal_daily['tchi'].interpolate(method='linear')
goyal_daily['shtint'] = goyal_daily['shtint'].interpolate(method='linear')

# For 'gdpce', fill each daily value with the corresponding end-of-month value for that month
goyal_daily['gdpce'] = goyal_daily['gdpce'].fillna(method='bfill')

# merge goyal data with the options data

df = df.merge(goyal_daily, left_index = True, right_index = True, how='left')




In [ ]:
df.to_csv('C:/Users/User/Desktop/Data speciale/NeuralNet_data_v2.csv')